## Import Data

In [7]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from error_generator import Random_Active_Domain
from error_generator import List_selected
from error_generator import Read_Write
from error_generator import Error_Generator
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LassoCV
from yellowbrick.regressor import AlphaSelection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

## Train-test dataset

In [2]:
anime_train = pd.read_csv("data/trainDataset.csv")
anime_test = pd.read_csv("data/testDataset.csv")

## Generating Errors

In [3]:
def error_generater(dataset, method, n, ignored_columns):

    myselector=List_selected()

    mygen=Error_Generator()

    new_dataset=mygen.error_generator(method_gen=method,selector=myselector,percentage=n,dataset=dataset,mute_column = ignored_columns)
    
    return new_dataset

In [8]:
# generate error in the training set
new_dataset = error_generater(anime_train.values.tolist(), Random_Active_Domain(),75, ignored_columns = [0,1,2,4,5,6,7])
Read_Write.write_csv_dataset("data/{}.csv".format(Random_Active_Domain().name), new_dataset)
anime_train = pd.read_csv("data/{}.csv".format(Random_Active_Domain().name), 
                            names = ['anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])
anime_train.to_csv("data/dirtyTrainDataset.csv", index = False,
                  columns = ['anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])

---------Change according to random_active_domain method ---------------

row: 1429 col: 3 : 'movie' changed to 'tv series'  
row: 165 col: 3 : 'movie' changed to 'movie'  
row: 138 col: 3 : 'tv series' changed to 'tv series'  
row: 92 col: 3 : 'tv series' changed to 'movie'  
row: 687 col: 3 : 'tv series' changed to 'tv series'  
row: 903 col: 3 : 'movie' changed to 'movie'  
row: 492 col: 3 : 'tv series' changed to 'tv series'  
row: 875 col: 3 : 'movie' changed to 'movie'  
row: 1416 col: 3 : 'tv series' changed to 'tv series'  
row: 1437 col: 3 : 'tv series' changed to 'tv series'  
row: 614 col: 3 : 'tv series' changed to 'movie'  
row: 1013 col: 3 : 'movie' changed to 'tv series'  
row: 818 col: 3 : 'movie' changed to 'tv series'  
row: 1454 col: 3 : 'movie' changed to 'tv series'  
row: 1151 col: 3 : 'tv series' changed to 'movie'  
row: 452 col: 3 : 'tv series' changed to 'movie'  
row: 1066 col: 3 : 'movie' changed to 'movie'  
row: 342 col: 3 : 'tv series' changed to 'movie' 

row: 996 col: 3 : 'tv series' changed to 'movie'  
row: 369 col: 3 : 'tv series' changed to 'tv series'  
row: 203 col: 3 : 'movie' changed to 'movie'  
row: 155 col: 3 : 'movie' changed to 'movie'  
row: 112 col: 3 : 'tv series' changed to 'movie'  
row: 1315 col: 3 : 'movie' changed to 'movie'  
row: 1568 col: 3 : 'movie' changed to 'tv series'  
row: 868 col: 3 : 'tv series' changed to 'tv series'  
row: 1435 col: 3 : 'movie' changed to 'movie'  
row: 463 col: 3 : 'movie' changed to 'movie'  
row: 665 col: 3 : 'movie' changed to 'tv series'  
row: 589 col: 3 : 'tv series' changed to 'tv series'  
row: 1229 col: 3 : 'tv series' changed to 'tv series'  
row: 989 col: 3 : 'movie' changed to 'tv series'  
row: 856 col: 3 : 'movie' changed to 'movie'  
row: 905 col: 3 : 'movie' changed to 'movie'  
row: 1584 col: 3 : 'movie' changed to 'tv series'  
row: 1087 col: 3 : 'tv series' changed to 'tv series'  
row: 117 col: 3 : 'movie' changed to 'tv series'  
row: 65 col: 3 : 'movie' changed 

## Transformation

In [9]:
import nltk
def get_words(x):
    bagofwords=[]
    for i in x:
        if i[1]=='NN':
            bagofwords.append(i[0])
        elif i[1]=='NNS':
            bagofwords.append(i[0])
        elif i[1]=='NNP':
            bagofwords.append(i[0])
        elif i[1]=='NNPS':
            bagofwords.append(i[0])
        elif i[1]=='JJ':
            bagofwords.append(i[0])
        elif i[1]=='JJR':
            bagofwords.append(i[0])
        elif i[1]=='JJS':
            bagofwords.append(i[0])
        elif i[1]=='RB':
            bagofwords.append(i[0])
        elif i[1]=='RBR':
            bagofwords.append(i[0])
        elif i[1]=='RBS':
            bagofwords.append(i[0])
    return bagofwords

def clean_words(x):
    b=nltk.pos_tag(nltk.word_tokenize(x))
    result=get_words(b)
    return result

In [10]:
def get_dummies(train_data_set, test_data_set):
    train_dummies = train_data_set.genre.str.get_dummies(',')
    test_dummies = test_data_set.genre.str.get_dummies(',')
    
    #print("Train Dummies",train_dummies.shape)    
    #print("Test Dummies",test_dummies.shape)
    
    #### ALİGN
    train_dummies, test_dummies = train_dummies.align(test_dummies, axis=1, join='left')
    
    test_dummies.fillna(0, inplace=True)
    
    type_lb = LabelBinarizer()
    fitted_type_lb = type_lb.fit(train_data_set.type.values)
    X_train = type_lb.transform(train_data_set.type.values)
    X_test  = type_lb.transform(test_data_set.type.values)
    
    dfOneHot_train = pd.DataFrame(X_train, columns = ["movie/TVseries" for i in range(X_train.shape[1])])
    dfOneHot_test  = pd.DataFrame(X_test,  columns = ["movie/TVseries" for i in range(X_test.shape[1])])
    
    
    train_data_set = pd.concat([train_data_set, dfOneHot_train], axis=1, join="inner")
    train_data_set = pd.concat([train_data_set, train_dummies ], axis=1, join="inner")

    test_data_set = pd.concat([test_data_set, dfOneHot_test], axis=1)
    test_data_set = pd.concat([test_data_set, test_dummies],  axis=1)
    
    test_data_set['movie/TVseries'].fillna(0, inplace=True)
    train_data_set['movie/TVseries'].fillna(0, inplace=True)

    return ([train_data_set, test_data_set])

def feature_transformation(train_data_set, test_data_set):
    
    dummieset = get_dummies(train_data_set, test_data_set)
    train_data_set = dummieset[0]
    test_data_set = dummieset[1]
        
    #Bag of Words
    summary_doc_train = train_data_set['overview'].fillna("").map(clean_words)
    summary_doc_train =summary_doc_train.apply(','.join)
    
    summary_doc_test = test_data_set['overview'].fillna("").map(clean_words)
    summary_doc_test =summary_doc_test.apply(','.join)
 
    vectorizer = TfidfVectorizer()
    fitted_vectorizer = vectorizer.fit(summary_doc_train)
    overview_feature_train = fitted_vectorizer.transform(summary_doc_train).toarray()
    overview_feature_test = fitted_vectorizer.transform(summary_doc_test).toarray()

    df_train = pd.DataFrame(overview_feature_train, columns = ["word"+ str(int(i)) for i in range(overview_feature_train.shape[1])])
    train_data_set = pd.concat([train_data_set, df_train], axis=1)
    
    df_test = pd.DataFrame(overview_feature_test, columns = ["word"+ str(int(i)) for i in range(overview_feature_test.shape[1])])
    test_data_set = pd.concat([test_data_set, df_test], axis=1)
    
    train_data_set = train_data_set.drop(columns=['anime_id', 'name', 'genre', 'overview', 'type'])
    test_data_set = test_data_set.drop(columns=['anime_id', 'name', 'genre', 'overview', 'type'])
    
    #drop NaN values
    train_data_set.dropna(inplace=True)
    test_data_set.dropna(inplace=True)
    
    train_data_set.fillna(0, inplace=True)
    test_data_set.fillna(0, inplace=True)
    
    return ([train_data_set, test_data_set])


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# features vector
transformed_features = feature_transformation(anime_train, anime_test)

anime_train = transformed_features[0]
anime_test = transformed_features[1]

print(anime_train.shape)
print(anime_test.shape)

# split train test
anime_y_train = anime_train['rating']
anime_X_train = anime_train.drop(columns=['rating'])

anime_y_test = anime_test['rating']
anime_X_test = anime_test.drop(columns=['rating'])


(1541, 7163)
(390, 7163)


## Feature Selection

In [12]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler

selector = SelectKBest(score_func=f_regression,k=700)#anime_X_test.shape[1]-1)
features = selector.fit(anime_X_train, anime_y_train)

# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)

print(anime_X_train.shape)
print(anime_X_test.shape)
anime_X_train = features.transform(anime_X_train)
anime_X_test = features.transform(anime_X_test)
print(anime_X_train.shape)
print(anime_X_test.shape)

scaler = StandardScaler() 

# Apply transform to both the training set and the test set.
anime_X_train = scaler.fit_transform(anime_X_train)  
anime_X_test = scaler.transform(anime_X_test) 

(1541, 7162)
(390, 7162)
(1541, 700)
(390, 700)


## Training - Testing - Evaluation

In [14]:
def training(model, dataset, label):
    clf = model
    clf.fit(dataset, label)
    return clf

def testing_evaluation(model, testset):
    # Make predictions using the testing set
    anime_y_pred = model.predict(testset)
    
    # The mean squared error
    print("Mean squared error: %.2f" % np.sqrt(mean_squared_error(anime_y_test, anime_y_pred)))


### Training

In [15]:
clf = training(model = linear_model.Lasso(), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)

Mean squared error: 1.05


### Cross Validation

In [16]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
scores = cross_val_score(clf, anime_X_train, anime_y_train, scoring="neg_mean_squared_error", cv=5) 
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)

Scores: [0.986 0.986 1.078 0.975 0.954]
Mean: 0.9959716665162077
Standard deviation: 0.042662998376204936


### Grid Search For Hyper Parameter Selection

In [17]:
def checkHP(model, folds, dataset, label):
    parameters = {
                    "alpha" : [1,0.1,0.01,0.001,0.0001,0]
                  }

    gd_sr = GridSearchCV(estimator=model,  
                         param_grid=parameters,
                         scoring="neg_mean_squared_error",
                         cv=folds)

    gd_sr.fit(dataset, label)  
    
    best_parameters = gd_sr.best_params_  
    print("best parameters are: ", best_parameters)

    best_result = gd_sr.best_score_  
    print("The mean squared Error is: %.2f" % -best_result) 
    
checkHP(clf, 5, anime_X_train, anime_y_train) 

best parameters are:  {'alpha': 0.01}
The mean squared Error is: 0.86


In [18]:
clf = training(model = linear_model.Lasso(alpha = 0.01), dataset = anime_X_train, label= anime_y_train)
testing_evaluation(clf, anime_X_test)

Mean squared error: 1.04
